# Introduction to Geovisualisation

This Jupyter notebook demonstrates how objects located at different places at different times can be displayed on a map.

The notebook can be run locally on the computer or in a cloud-based environment such as Google Colab. When using Google Colab, this notebook can be accessed directly using the following URL: https://colab.research.google.com/github/history-unibas/economies-of-space-teaching-summer-school-ss2025/blob/main/introduction_to_geovisualisation.ipynb

## Load necessary modules 

In [9]:
import pandas as pd
import os
import subprocess


from ipyleaflet import Map, basemaps, WMSLayer, Marker, AntPath, Popup, FullScreenControl
from ipywidgets import HTML, VBox

## Connect personal Google Drive with Google Colab

In [10]:
try:
    from google.colab import drive
    in_colab = True
    
except ImportError:
    in_colab = False

if in_colab:
  
  # Mount personal google drive.
  drive.mount('/content/drive')

## Choose a base map

### Base maps of the ipyleaflet module

More information: https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html

In [11]:
# Define the centre of the map (lat, lon).
MAP_CENTRE = (46.950, 7.445)

# Define the zoom level of the map.
ZOOM_LEVEL = 12

m1 = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=MAP_CENTRE, zoom=ZOOM_LEVEL)

m1

Map(center=[46.95, 7.445], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Base maps from the Federal Office of Topography swisstopo

Examples of layers:
- ch.swisstopo.pixelkarte-farbe
- ch.swisstopo.pixelkarte-grau
- ch.swisstopo.swissimage

In [12]:
wms_layer = WMSLayer(
    url = 'https://wms.geo.admin.ch/',
    layers='ch.swisstopo.pixelkarte-farbe',
    format='image/png',
    attribution='© swisstopo'
)
m2 = Map(basemap=wms_layer, center=MAP_CENTRE, zoom=ZOOM_LEVEL)

m2

Map(center=[46.95, 7.445], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

## Import data for visualisation

We use a CSV file as the basis for displaying objects on the map. The following columns are required for display on the map:
| Name of the column   | Description          |
|----------|----------------------|
| lon | Longitude coordinate in the coordinate system EPSG:4326 (World Geodetic System 1984, WGS 84) |
| lat | Latitude coordinate in the coordinate system EPSG:4326 (World Geodetic System 1984, WGS 84) |
| datetime | The date and time at which the object was located at this time |
| description | Description of the location |
| object_id | Unique key of the object |

In [13]:
# Filename of the input data.
FILENAME_DATA = 'example_data.csv'

# URL of the input data.
URL_DATA = 'https://raw.githubusercontent.com/history-unibas/economies-of-space-teaching-summer-school-ss2025/refs/heads/main/example_data.csv'

# Working directory in Google Colab.
WORKING_DIRECTORY = '/content/drive/My Drive/Colab Notebooks/'

# Save the data in google drive.
if in_colab:
    
    if not os.path.exists(WORKING_DIRECTORY + FILENAME_DATA):
        
        # Download the data in working directory.
        subprocess.run(['wget', URL_DATA, '-O', WORKING_DIRECTORY + FILENAME_DATA], check=True)
      
    # Update the filepath.
    FILENAME_DATA = WORKING_DIRECTORY + FILENAME_DATA

In [14]:
# Load the data.
df = pd.read_csv(FILENAME_DATA, parse_dates=['datetime'])

display(df)

,lon,lat,fid,datetime,description,object_id
0,7.430115,46.953651,1,2025-06-24 09:00:00,Uni Tobler,1
1,7.437048,46.950934,2,2025-06-24 12:00:00,State Archives of Bern,1
2,7.430115,46.953651,3,2025-06-24 16:00:00,Uni Tobler,1
3,7.430115,46.953651,4,2025-06-25 09:00:00,Uni Tobler,2
4,7.430115,46.953651,5,2025-06-25 12:00:00,Uni Tobler,2
5,7.439477,46.919316,6,2025-06-25 19:00:00,Gurten,2
6,7.430115,46.953651,7,2025-06-26 09:00:00,Uni Tobler,3
7,7.449295,46.942984,8,2025-06-26 12:00:00,Bern Historical Museum,3
8,7.430115,46.953651,9,2025-06-26 16:00:00,Uni Tobler,3


## Show data on the map

In [15]:
# Shift of the lines in degree.
SHIFT = 0.0001

# Create a map for each object.
maps = []
grouped = df.groupby('object_id')
for _, group in grouped:
    
    # Define a map.
    MAP_CENTRE = (group['lat'].mean(), group['lon'].mean())
    m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=MAP_CENTRE, zoom=ZOOM_LEVEL)

    group_sorted = group.sort_values(by='datetime')
    coords_prev = ()
    
    for _, entry in group_sorted.iterrows():
        
        # Add markers with popup to the map.
        marker = Marker(location=(entry['lat'], entry['lon']))
        popup = Popup(location=(entry['lat'], entry['lon']), child=HTML(entry['description']), close_button=True)
        marker.popup = popup        
        m.add_layer(marker)

        coords_current = (entry['lat'], entry['lon'])
        if coords_prev:
            
            # Shift the coordinates.
            if coords_current[0] < coords_prev[0]:
                coords_prev_shifted = (coords_prev[0] + SHIFT, coords_prev[1] + SHIFT)
                coords_current_shifted = (coords_current[0] + SHIFT, coords_current[1] + SHIFT)
            else:
                coords_prev_shifted = (coords_prev[0] - SHIFT, coords_prev[1] - SHIFT)
                coords_current_shifted = (coords_current[0] - SHIFT, coords_current[1] - SHIFT)

            # Create a line between both points.
            arrow_line = AntPath(locations=[coords_prev_shifted, coords_current_shifted],
                                 delay=4000,
                                 weight=4
                                 )
            m.add_layer(arrow_line)
            
        coords_prev = tuple(coords_current)

    maps.append(m)

# Show the maps.
vbox = VBox(maps)
display(vbox)

## Export the map as html file

In [16]:
# File name of the html to be exported.
FILENAME_MAP = 'map.html'

# Export the first map as an example.
m3 = maps[0]
m3.add_control(FullScreenControl())

if in_colab:
    m3.save(WORKING_DIRECTORY + FILENAME_MAP)

else:
    m3.save(FILENAME_MAP)